# Multi-objective robust decision making (MORDM)


This exercise demostrates the application of MORDM on the lake model, which was used in earlier exercises.

MORDM has four main steps:

(i)	    **problem formulation** based on a systems analytical problem definition framework 

(ii)	**searching** for candidate solutions that optimize multiple objectives by using multi-objective evolutionary algorithms 

(iii)	generating an ensemble of scenarios to **explore** the effects of uncertainties 

(iv)	using **scenario discovery** to detect the vulnerabilities of candidate solutions and improving thecandidate solutions



## Step 1: Problem formulation
### Lake Model

Remember the lake problem used in the assignments in previous weeks. The lake problem is a hypothetical case where the inhabitants of a lake town decide on the amount of annual pollution they release into a lake. It the pollution in the lake passes a threshold, it will suffer irreversible eutrophication.

The lake problem has 4 **outcome indicators**: 
   - **max_P**: maximum pollution over time, to be minimized
   - **utility**: economic benefits obtained from polluting the lake, to be maximized
   - **inertia**: the percentage of significant annual changes in the anthropogenic pollution rate, to be maximized
   - **reliability**: the percentage of years where the pollution level is below the critical threshold, to be maximized
    
See the lake model exercise for the formulation of these outcome variables.

The lake problem is characterized by both stochastic uncertainty and **deep uncertainty**. The stochastic uncertainty arises from the natural inflow. To reduce this stochastic uncertainty, multiple replications are performed and the average over the replication is taken. Deep uncertainty is presented by uncertainty about the mean $\mu$ and standard deviation $sigma$ of the lognormal distribution characterizing the natural inflow, the natural removal rate of the lake $\beta$, the natural recycling rate of the lake $q$, and the discount rate $\delta$. The table below specifies the ranges for the deeply uncertain factors, as well as their best estimate or default values. 

|Parameter	|Range	        |Default value|
|-----------|--------------:|------------:|
|$\mu$    	|0.01 – 0.05	|0.02         |
|$\sigma$	|0.001 – 0.005 	|0.0017       |
|$b$      	|0.1 – 0.45	    |0.42         |
|$q$	    |2 – 4.5	    |2            |
|$\delta$	|0.93 – 0.99	|0.98         |


The lake problem in previous assignments had 100 decision **levers**, meaning that the decision makers independently decide on the amount of anthropogenic pollution at every time step (100). Then a 'policy' was a set of values for these 100 levers, which you composed by sampling from the range [0, 0.1].   

In this exercise, we will use a more advanced way of deciding on the amout of anhtropogenic polution. We will use a **closed loop** version of the lake model, meaning that $a_t$ (anthropogenic pollution) is dependent on $X_t$ (the pollution level at time t). For instance, the rate of anthropogenic pollutions is lowered if the pollution level is approaching a critical threshold. Here, we use "cubic radial basis functions" following [Quinn et al. 2017](http://www.sciencedirect.com/science/article/pii/S1364815216302250) and formulate $a_t$ as follows:

\begin{equation}
    a_{t} =  min\Bigg(max\bigg(\sum\limits_{j=1}^{n} w_{j}\left\vert{\frac{X_{t,i}-c_{j}}{r_{j}}}\right\vert^3, 0.01\bigg), 0.1\Bigg) \\
    s.t. \\
    -2 \leq c_{j} \leq 2 \\
    0 \leq r_{j} \leq 2 \\ 
    0 \leq w_{j} \leq 1 \\
    \sum\limits_{j=1}^{n} w_{j} = 1
\end{equation}


The parameters that define this function also define the pollution strategy over time. Hence, the decision **levers** are the five parameters of this functions, namely $c_1$, $c_2$, $r_1$, $r_2$ and $w_1$. ($w_2$ = 1 - $w_1$).

Note:: i is index for the realization, given m realizations; j is the index for the radial basis function, given 2 radial basis functions. 

**To formulate this problem, do the following:**

**1) Import the lake model function from dps_lake_model.py**

**2) Create an ema_workbench interface for this problem, with corresponding uncertainties, levers and outcomes as specified above**



In [1]:
import pandas as pd

from Week_5_6_quaq.dps_lake_model import lake_model as lake_problem

from ema_workbench import (Model, RealParameter, ScalarOutcome, Constant, SequentialEvaluator, MultiprocessingEvaluator,
                           ema_logging, Constraint, Policy, save_results)
from ema_workbench.analysis import (parcoords, prim)
import matplotlib.pyplot as plt


lake_model = Model('lake_model', function=lake_problem)
lake_model.time_horizon = 100

lake_model.uncertainties = [RealParameter("mean", 0.01, 0.05),
                            RealParameter("stdev", 0.001, 0.005),
                            RealParameter("b", 0.1, 0.45),
                            RealParameter("q", 2, 4.5),
                            RealParameter("delta", 0.93, 0.99)]

lake_model.levers = [RealParameter("c1", -2, 2),
                     RealParameter("c2", -2, 2),
                     RealParameter("r1", 0, 2),
                     RealParameter("r2", 0, 2),
                     RealParameter("w1", 0, 1)]

lake_model.outcomes = [ScalarOutcome("max_P", kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome("utility", kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome("inertia", kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome("reliability", kind=ScalarOutcome.MAXIMIZE)]

n_scenarios = 100

with SequentialEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios)


ModuleNotFoundError: No module named 'ema_workbench'

## Step 2: Searching for candidate solutions

In the second step of MORDM, candidate strategies are identified which are pareto optimal conditional on a reference scenario. These candiate strategies are identified through search with multi-objective evolutionary algorithms, that iteratively evaluate a large number of alternatives on multiple objectives until they find the best candidates. For instance, when we optimize the lake model levers, the lake model function will be called for each candidate evaluation, and the corresponding four objective values will be generated. 

Take the model interface developed in the previous step and use the optimization functionality of the workbench to identify the pareto approximate set of solutions. Try the following:
* change the epsilon values between 0.01 and 0.1, what changes, why?
* change the number of function evaluations from 1000 to 10.000 (this requires using multiprocessing unless you are very patient). What is the difference? You can use  convergence as explained in assignment 7 for this

In [1]:
nfes = [1000, 10000]
epsilons = [0.01, 0.02, 0.05, 0.1]
for nfe in nfes:
    for epsilon in epsilons:
        epsilon = [epsilon, epsilon, epsilon, epsilon]
        with SequentialEvaluator(lake_model) as evaluator:
            results = evaluator.optimize(epsilons=epsilon, nfe=nfe)
        outcomes = results.loc[:, ['max_P', 'utility', "inertia", 'reliability']]
        limits = analysis.parcoords.get_limits(outcomes)
        axes = analysis.parcoords.ParallelAxes(limits)
        axes.plot(outcomes)

        # we invert this axis so direction of desirability is the same 
        axes.invert_axis('max_P') 
        plt.show()


NameError: name 'SequentialEvaluator' is not defined

### Different epsilons and nfe's
The different epsilon values give different sets of optimal solution. The higher the epsilon values, the less final solutions are given. However, it can be seen from the plots that these solutions are distributed in a similar manner. 
Increasing the nfe's on the other hand, does not necessarily increase or decrease the number of final optimal solution, but the plot does suggest that the solutions that have been found are distributed more equally across each of the three parameters. 

**plot the tradeoffs you have found using a parallel axis plot**

We can visualize these tradeoffs on **parallel axis plots**. In these plots, each dimension is shown as a vertical axis. Each solution is represented by a line on this plot, which crosses the objective axes at the corresponsing value. You can use the [parcoords functionality](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/parcoords.html) for this that comes with the ema_workbench. Ensure that the direction of desirability is the same for the four objectives.|



**What does this plot tell us about the tradeoffs and conflicting objectives?**

## See plots above.

## Step 3: Re-evaluate candidate solutions under uncertainty

We now have a large number of candidate solutions (policies), we can re-evaluate them over the various deeply uncertain factors to assess their robustness against uncertainties.

For this robustness evaluation, we need to explore the scenarios for each solution. It means that, if we would like to run for instance 1000 scenarios for each solution, we might have to execute a very large number of runs.

Here, to simplify the case, let's suppose that decision makers have a hard constrain on *reliability*. No solution with less than 90% reliability is acceptable for them. Therefore, we can reduce the size of the solution set according to this constraint. 

**Apply this constraint of reliability on the results, and create a new dataframe named new_results**


#### New results
There are two ways in which one can obtain the dataframe for the new results. One way is to simply run the old version and filter out only the rows where the reliability is higher than 90%. Another way is to apply a constraint through the `Constraints` function in the `ema_workbench`. Since the exercise did not ask to run the full model again with the constraints variable applied, the first methodology has been chosen. However, if one wanted to apply the second methodolgy, the constraint code would look like the following:
```python
constraints = [Constraint("reliability", 
                          outcome_names="reliability", 
                          function=lambda x : max(0.9, 1.8-x))]

            with SequentialEvaluator(lake_model) as evaluator:
                new_results = evaluator.optimize(epsilons=epsilon, 
                                                 nfe=nfe, 
                                                 constraints=constraints)
```

The code above will yield a new dataframe called `new_results`, where the results will only include those where the reliability is higher than 0.9.

In [ ]:
new_results = results.loc[results["reliability"] >= 0.9]


**From new_results, which is the reduced dataframe of candidate solutions, make a list of policies in a format that can be inputed to the *perform_experiments* function of the EMA workbench.**

*hint: you need to transform each policy to a dict, and then use this dict as input for the Policy class that comes with the workbench*

In [22]:
policy_results = new_results.drop(['max_P', 'utility', "inertia", 'reliability'], axis=1)
new_results_dict = new_results.to_dict('index')
policies = []
for dict_index in range(len(new_results_dict)):
    policy_dict = new_results_dict[dict_index]
    policies.append(Policy(f"pol_{dict_index}", **policy_dict))

**Perform 1000 scenarios for each of the policy options. Depending on how many solutions are left after implementing the constraint, consider using multiprocessing or ipyparallel to speed up calculations.**

If you want to use ipyparallel, don't forget to start ipcluster.

In [11]:
n_scenarios = 1000
with SequentialEvaluator(lake_model) as evaluator:
    results_w_policy = evaluator.perform_experiments(n_scenarios, 
                                                     policies=policies)
results_w_policy.rename(columns={"max_P": "max_P_initial",
                                 "inertia": "inertia_initial",
                                 "utility": "utility_initial",
                                 "reliability": "reliability_initial"},
                        inplace=True)

df_outcomes_w_policy = pd.DataFrame(outcomes_w_policy)

df_full = pd.concat([df_outcomes_w_policy, results_w_policy], axis=1)

We can now evaluate the **robustness** of each of the policy options based on these scenario results. We can calculate the robustness of a policy option in terms of its performance on an outcome indicator across the 1000 scenarios. In other words, we can identify how robust a policy is in terms of each outcome indicator, and investigate the robustness tradeoffs.  

There are multiple metrics to quantify robustness. On of them is the *signal to noise ratio*, which is simply the mean of a dataset divided by its standard deviation. For instance, for an outcome indicator to be maximized, we prefer a high average value across the scenarios, and a low standard deviation, implying a narrow uncertaintiy range. Therefore, we want to maximize the signal-to-noise ratio. For an outcome indicator to be minimized, a lower mean and a lower standard deviation is preferred. Therefore the formulation is different.

**Write a function to calculate the signal-to-noise ratio for both kinds of outcome indicators. Calculate the signal-to-noise ratios for each outcome and each policy option. Plot the tradeoffs on a parallel axis plot. Which solutions look like a good compromise policy?**

In [ ]:
df_outcomes_w_policy = pd.DataFrame(outcomes_w_policy)

signal_noise_r = []
for policy_analyzed in range(len(policies)):
    df = (df_outcomes_w_policy.tail((len(policies) - policy_analyzed)*n_scenarios)).head(n_scenarios)
    df.loc["mean"] = df.mean()
    df.loc["stdev"] = df.std()
    df.loc["ratio"] = df.loc["mean"] / df.loc["stdev"]
    partial_dict = df.loc["ratio"].to_dict()
    signal_noise_r.append(partial_dict)
    
df_plot_signals = pd.DataFrame(signal_noise_r)

limits = parcoords.get_limits(df_plot_signals)
axes = parcoords.ParallelAxes(limits)
axes.plot(df_plot_signals)
plt.show()

Another robustness metric is **maximum regret**, calculated again for each policy and for each outcome indicator. *Regret* is defined for each policy under each scenario, as the difference between the performance of the policy in a specific scenario and the berformance of a no-regret (i.e. best possible result in that scenario) policy. The *maximum regret*  is then the maximum of such regret values across all scenarios. We of course favor policy options with low *maximum regret* values. 

**Write a function to calculate the maximum regret. Calculate the maximum regret values for each outcome and each policy option. Plot the tradeoffs on a parallel plot. Which solutions look like a good compromise policy?**

In [ ]:
def calculate_regret() :
    df_regret = pd.DataFrame(columns=['policy_num', 'max_P', 'utility', "inertia", 'reliability'])
    for unique_item in df_full['policy'].unique() :
        df_partial = df_full.loc[df_full['policy'] == unique_item]
        df_partial.reset_index(drop=True, inplace=True)
        max_max_P = max(df_partial['max_P'])
        max_utility = max(df_partial['utility'])
        max_inertia = max(df_partial['inertia'])
        max_reliability = max(df_partial['reliability'])
        for index in range(len(df_partial)) :
            new_row = {'policy_num' : unique_item,
                       'max_P' : max_max_P - df_partial.loc[index, 'max_P'],
                       'utility' : max_utility - df_partial.loc[index, 'utility'],
                       'inertia' : max_inertia - df_partial.loc[index, 'inertia'],
                       'reliability' : max_reliability - df_partial.loc[index, 'reliability']}
            df_regret = df_regret.append(new_row, ignore_index=True)

    columns_to_sum = ['max_P', 'utility', "inertia", 'reliability']
    df_regret['total_regret'] = df_regret[columns_to_sum].sum(axis=1)

    return df_regret
df_regret = calculate_regret()

df_regret_plot = pd.DataFrame(columns=['policy', 'max_regret', 'min_regret'])
for unique_item in df_regret['policy_num'].unique() :
    df_partial = df_regret.loc[df_regret['policy_num'] == unique_item]
    new_row = {'policy' : unique_item,
               'max_regret' : max(df_partial['total_regret']),
               'min_regret' : min(df_partial['total_regret'])}
    df_regret_plot = df_regret_plot.append(new_row, ignore_index=True)

In [ ]:
df_regret_plot.sort_values(['max_regret'], inplace=True)
best_policy = df_regret_plot.loc[0, 'policy']
best_c1 = results_w_policy.loc[results_w_policy['policy'] == best_policy, "c1"].iloc[0]
best_c2 = results_w_policy.loc[results_w_policy['policy'] == best_policy, "c2"].iloc[0]
best_r1 = results_w_policy.loc[results_w_policy['policy'] == best_policy, "r1"].iloc[0]
best_r2 = results_w_policy.loc[results_w_policy['policy'] == best_policy, "r2"].iloc[0]
best_w1 = results_w_policy.loc[results_w_policy['policy'] == best_policy, "w1"].iloc[0]
print(f"Best policy is policy number: {best_policy}\n"
      f"This policy has the following values: \n"
      f"c1: {best_c1}\n"
      f"c2: {best_c2}\n"
      f"r1: {best_r1}\n"
      f"r2: {best_r2}\n"
      f"w1: {best_w1}")

We now have an understanding of which solutions have decent robustness using 2 different robustness metrics. A related but different question is to assess the uncertain conditions under which we get poor performance. For this, we can use scenario discovery. Since we want to identify the uncertainties only, we can remove the policy column from the experiments DataFrame. 

**Perform Scenario Discovery, focussed on understanding the conditions under which utility is lower than 0.25**

from the trade off curve between coverage, density and number of restricted dimensions, select a point which balances them. Next, using the `yi` attribute, select from the experiments data frame the rows which are within the box as well as the outcomes associated with these experiments.Save these results. They are the starting point for the next assignment. In pseudo code:

```python
from ema_workbench import save_results

selected_experiments = experiments.iloc[box.yi]
selected_outcomes = {k:v[box.yi] for k,v in outcomes.items()}

save_results((selected_experiments, selected_outcomes), './results/selected_results.tar.gz')
```





In [ ]:
x = results_w_policy
y = outcomes_w_policy['utility'] < 0.25
prim_alg = prim.Prim(x, y, threshold=0.8)
box1 = prim_alg.find_box()
box1.show_tradeoff()
plt.show()

In [ ]:
selected_experiments = results_w_policy.iloc[box1.yi]
selected_outcomes = {k : v[box1.yi] for k, v in results_w_policy.items()}

save_results((selected_experiments, selected_outcomes), './results/selected_results.tar.gz')